In [15]:
import pandas as pd

# Read the CSV file
User = pd.read_csv('EVIO_history_01-06-2022_31-05-2023.csv', sep=";")

# Select required columns
User = User[['Start date','Stop date','Duration (min)', 'Total Energy (kWh)']]

# Convertendo os valores de energia para o formato numérico adequado
User['Total Energy (kWh)'] = User['Total Energy (kWh)'].str.replace(',', '.').astype(float)
User['Duration (min)'] = User['Duration (min)'].str.replace(',', '.').astype(float)

# Remove rows where 'Total Energy (kWh)' is less than 1 kWh
User = User[User['Total Energy (kWh)'] >= 1]

# Remove rows where 'Duration (min)' is less than 5 minutes
User = User[User['Duration (min)'] >= 5]

#print(User)


# Convertendo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'], format='%m/%d/%Y | %H:%M')
User['Stop date'] = pd.to_datetime(User['Stop date'], format='%m/%d/%Y | %H:%M')


# Criando uma nova coluna 'Weekday' que contém o dia da semana
User['Weekday'] = User['Start date'].dt.day_name()

#print(User)

weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

User['Weekday'] = User['Weekday'].map(weekday_mapping)





# Converter as colunas 'Start date' e 'Stop date' para o tipo datetime, se ainda não foram convertidas
User['Start date'] = pd.to_datetime(User['Start date'])
User['Stop date'] = pd.to_datetime(User['Stop date'])

# Criar colunas para cada hora do dia (8h às 20h)
hours = range(8, 21)
for hour in hours:
    User[f'Charging_{hour}h'] = 0

# Preencher as colunas com valores binários (1 ou 0)
for index, row in User.iterrows():
    start_hour = row['Start date'].hour
    stop_hour = row['Stop date'].hour
    for hour in range(8, 21):
        if hour >= start_hour and hour <= stop_hour:
            User.at[index, f'Charging_{hour}h'] = 1




# Converter as colunas de data para o formato datetime
User["Start date"] = pd.to_datetime(User["Start date"], format="%m/%d/%Y | %H:%M")
User["Stop date"] = pd.to_datetime(User["Stop date"], format="%m/%d/%Y | %H:%M")

# Ordenar o dataframe pela coluna 'Start date' para garantir que os dados estejam em ordem temporal
User.sort_values(by='Start date', inplace=True)

# Calcular a diferença entre o stop date do último carregamento e o start date da linha seguinte em dias
User['Days_since_last_charge'] = (User['Start date'] - User['Stop date'].shift(1)).dt.days
# Preencher o valor NaN na primeira linha com 0, já que não houve carregamento anterior
User['Days_since_last_charge'].fillna(0, inplace=True)

# Calcular a diferença entre o 'Start date' da linha atual e o 'Stop date' da linha anterior em horas
User['Hours_since_last_charge'] = (User['Start date'] - User['Stop date'].shift(1)).dt.total_seconds() / 3600
# Preencher o valor NaN na primeira linha com 0, já que não houve carregamento anterior
User['Hours_since_last_charge'].fillna(0, inplace=True)

# Exibir o dataframe atualizado
print(User)

             Start date           Stop date  Duration (min)  \
116 2022-06-07 13:52:00 2022-06-07 17:36:00         224.517   
115 2022-06-15 15:08:00 2022-06-15 15:51:00          43.467   
114 2022-06-15 18:37:00 2022-06-15 18:49:00          12.100   
113 2022-06-15 18:51:00 2022-06-15 18:57:00           6.167   
108 2022-07-19 15:23:00 2022-07-19 20:50:00         327.283   
..                  ...                 ...             ...   
5   2023-05-08 08:38:00 2023-05-08 12:58:00         260.017   
4   2023-05-24 18:42:00 2023-05-24 19:17:00          35.300   
3   2023-05-26 18:51:00 2023-05-26 19:13:00          22.550   
2   2023-05-30 09:32:00 2023-05-30 13:29:00         237.100   
0   2023-05-31 13:46:00 2023-05-31 19:08:00         321.983   

     Total Energy (kWh)  Weekday  Charging_8h  Charging_9h  Charging_10h  \
116              40.190        2            0            0             0   
115              41.180        3            0            0             0   
114            

In [16]:
import pandas as pd

# Supondo que o dataframe User já exista
# User = pd.read_csv('EVIO_history_01-06-2022_31-05-2023.csv', sep=";")

# Definindo as colunas de data para o formato de data especificado
User['Start date'] = pd.to_datetime(User['Start date'])
User['Stop date'] = pd.to_datetime(User['Stop date'])

# Gerando um índice de datas para o intervalo específico
start_date = "2022-06-07"
end_date = "2023-05-31"
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
full_dates_df = pd.DataFrame(date_range, columns=['Date'])

# Converter a coluna 'Start date' do User para o mesmo formato de data
User['Date'] = User['Start date'].dt.normalize()

# Unir este índice ao dataset existente
full_dataset = full_dates_df.merge(User, on='Date', how='left')

# Preencher campos com zero para dias sem carregamento
columns_to_fill = ['Duration (min)', 'Total Energy (kWh)', 'Weekday', 'Charging_8h', 'Charging_9h', 
                   'Charging_10h', 'Charging_11h', 'Charging_12h', 'Charging_13h', 'Charging_14h', 
                   'Charging_15h', 'Charging_16h', 'Charging_17h', 'Charging_18h', 'Charging_19h', 
                   'Charging_20h', 'Days_since_last_charge', 'Hours_since_last_charge']

full_dataset[columns_to_fill] = full_dataset[columns_to_fill].fillna(0)

# Corrigir a coluna Weekday
weekday_mapping = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}
full_dataset['Weekday'] = full_dataset['Date'].dt.day_name().map(weekday_mapping)

# Preencher as colunas 'Days_since_last_charge' e 'Hours_since_last_charge' corretamente
for i in range(1, len(full_dataset)):
    if full_dataset.loc[i, 'Total Energy (kWh)'] == 0:
        full_dataset.loc[i, 'Days_since_last_charge'] = full_dataset.loc[i-1, 'Days_since_last_charge'] + 1
        full_dataset.loc[i, 'Hours_since_last_charge'] = full_dataset.loc[i-1, 'Hours_since_last_charge'] + 24
    else:
        full_dataset.loc[i, 'Days_since_last_charge'] = 0
        full_dataset.loc[i, 'Hours_since_last_charge'] = 0

# Exibir o dataframe atualizado
print(full_dataset)

          Date          Start date           Stop date  Duration (min)  \
0   2022-06-07 2022-06-07 13:52:00 2022-06-07 17:36:00         224.517   
1   2022-06-08                 NaT                 NaT           0.000   
2   2022-06-09                 NaT                 NaT           0.000   
3   2022-06-10                 NaT                 NaT           0.000   
4   2022-06-11                 NaT                 NaT           0.000   
..         ...                 ...                 ...             ...   
374 2023-05-27                 NaT                 NaT           0.000   
375 2023-05-28                 NaT                 NaT           0.000   
376 2023-05-29                 NaT                 NaT           0.000   
377 2023-05-30 2023-05-30 09:32:00 2023-05-30 13:29:00         237.100   
378 2023-05-31 2023-05-31 13:46:00 2023-05-31 19:08:00         321.983   

     Total Energy (kWh)  Weekday  Charging_8h  Charging_9h  Charging_10h  \
0                 40.19        2   

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Supondo que o dataframe full_dataset já exista

# Usar 'Weekday' e 'Hours_since_last_charge' como variáveis de entrada
X = full_dataset[['Weekday', 'Hours_since_last_charge']].values
y = full_dataset[[f'Charging_{hour}h' for hour in range(8, 21)]]

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo comparando as previsões com y_test
accuracy = accuracy_score(y_test.values.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in range(8, 21)])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.values.ravel(), predictions.ravel())

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100 if total_positives > 0 else 0
percentage_true_negatives = overall_true_negatives / total_negatives * 100 if total_negatives > 0 else 0

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')


Acurácia do modelo: 0.917004048582996
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar      Carregar      Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..           ...           ...           ...           ...           ...   
71  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
72  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
73  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
74  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
75  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

    Charging_13h  Charging_14h  Chargi

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Supondo que o dataframe full_dataset já exista

# Usar 'Weekday' e 'Hours_since_last_charge' como variáveis de entrada
X = full_dataset[['Weekday', 'Hours_since_last_charge']].values
y = full_dataset[[f'Charging_{hour}h' for hour in range(8, 21)]]

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo comparando as previsões com y_test
accuracy = accuracy_score(y_test.values.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in range(8, 21)])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões
print("Previsões:")
print(predictions_table)

# Calcular as matrizes de confusão para cada hora
hours = range(8, 21)
true_positive_rates = {}
true_negative_rates = {}

for hour in hours:
    y_true_hour = y_test[f'Charging_{hour}h']
    y_pred_hour = predictions[:, hours.index(hour)]
    
    # Calcular a matriz de confusão para a hora atual
    cm = confusion_matrix(y_true_hour, y_pred_hour)
    
    # Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão
    true_positives = cm[1, 1] if cm.shape[0] > 1 else 0
    true_negatives = cm[0, 0] if cm.shape[0] > 0 else 0
    
    # Calcular o total de exemplos positivos e negativos
    total_positives = cm[:, 1].sum() if cm.shape[1] > 1 else 0
    total_negatives = cm[:, 0].sum() if cm.shape[1] > 0 else 0
    
    # Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
    percentage_true_positives = (true_positives / total_positives * 100) if total_positives > 0 else 0
    percentage_true_negatives = (true_negatives / total_negatives * 100) if total_negatives > 0 else 0
    
    # Armazenar as percentagens no dicionário
    true_positive_rates[hour] = percentage_true_positives
    true_negative_rates[hour] = percentage_true_negatives

# Imprimir os resultados
for hour in hours:
    print(f'Hour {hour}: Percentagem de True Positives: {true_positive_rates[hour]:.2f}%')
    print(f'Hour {hour}: Percentagem de True Negatives: {true_negative_rates[hour]:.2f}%')


Acurácia do modelo: 0.917004048582996
Previsões:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar      Carregar      Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..           ...           ...           ...           ...           ...   
71  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
72  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
73  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
74  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
75  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

    Charging_13h  Charging_14h  Chargi

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Supondo que o dataframe full_dataset já exista

# Usar 'Weekday' e 'Hours_since_last_charge' como variáveis de entrada
X = full_dataset[['Weekday', 'Hours_since_last_charge']].values
y = full_dataset[[f'Charging_{hour}h' for hour in range(8, 21)]]

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# Fazer previsões para o conjunto de teste
predictions = model.predict(X_test)

# Avaliar a precisão do modelo comparando as previsões com y_test
accuracy = accuracy_score(y_test.values.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)


# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in range(8, 21)])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões para o próximo dia
print("Previsões para o próximo dia:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.values.ravel(), predictions.ravel())
#print(overall_confusion_matrix)

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100
percentage_true_negatives = overall_true_negatives / total_negatives * 100

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.917004048582996
Previsões para o próximo dia:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar      Carregar      Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
..           ...           ...           ...           ...           ...   
71  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
72  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
73  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
74  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
75  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   

    Charging_13h  C

In [13]:
# Extrair a coluna da hora das 13h dos dados de teste
y_true_13h = y_test['Charging_13h']

# Contar quantos valores "Carregar" (1) existem na coluna
carregar_real = y_true_13h.sum()

print(f"Número real de vezes que a bateria precisou ser carregada às 13h: {carregar_real}")

# Extrair a coluna da hora das 13h das previsões
y_pred_13h = predictions_table['Charging_13h']

# Contar quantos valores "Carregar" (1) existem na coluna
carregar_previsto = (y_pred_13h == 'Carregar').sum()

print(f"Número de vezes que o modelo previu 'Carregar' às 13h: {carregar_previsto}")

Número real de vezes que a bateria precisou ser carregada às 13h: 9.0
Número de vezes que o modelo previu 'Carregar' às 13h: 0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Preparar os dados de treinamento
X = User[['Weekday', 'Hours_since_last_charge']].values  # Usar 'Weekday' e 'Days_since_last_charge' como variáveis de entrada
y = User[[f'Charging_{hour}h' for hour in range(8, 21)]]

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)  # Adicionando shuffle=False para evitar misturar os dados

# Treinar o modelo de regressão logística multinomial
model = MultiOutputClassifier(LogisticRegression(max_iter=1000))  # Usar MultiOutputClassifier para lidar com múltiplas saídas
model.fit(X_train, y_train)

# Fazer previsões para o próximo dia usando X_test
predictions = model.predict(X_test)

# Avaliar a precisão do modelo comparando as previsões com y_test
accuracy = accuracy_score(y_test.values.ravel(), predictions.ravel())
print("Acurácia do modelo:", accuracy)

# Criar uma tabela de previsões com 0s e 1s
predictions_table = pd.DataFrame(predictions, columns=[f'Charging_{hour}h' for hour in range(8, 21)])
predictions_table.replace({0: 'Não Carregar', 1: 'Carregar'}, inplace=True)

# Exibir as previsões para o próximo dia
print("Previsões para o próximo dia:")
print(predictions_table)

# Calcular a matriz de confusão geral
overall_confusion_matrix = confusion_matrix(y_test.values.ravel(), predictions.ravel())
#print(overall_confusion_matrix)

# Extrair os verdadeiros positivos e verdadeiros negativos da matriz de confusão geral
overall_true_positives = overall_confusion_matrix[1, 1]
overall_true_negatives = overall_confusion_matrix[0, 0]

# Calcular o total de exemplos positivos e negativos
total_positives = overall_confusion_matrix[:, 1].sum()  # Soma da segunda coluna
total_negatives = overall_confusion_matrix[:, 0].sum()  # Soma da primeira coluna

# Calcular as percentagens de verdadeiros positivos e verdadeiros negativos
percentage_true_positives = overall_true_positives / total_positives * 100
percentage_true_negatives = overall_true_negatives / total_negatives * 100

# Imprimir os resultados
print(f'Percentagem de True Positives: {percentage_true_positives:.2f}%')
print(f'Percentagem de True Negatives: {percentage_true_negatives:.2f}%')

Acurácia do modelo: 0.6959706959706959
Previsões para o próximo dia:
     Charging_8h   Charging_9h  Charging_10h  Charging_11h  Charging_12h  \
0   Não Carregar  Não Carregar  Não Carregar      Carregar  Não Carregar   
1   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
2   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
3   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
4   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
5   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
6   Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
7   Não Carregar      Carregar  Não Carregar  Não Carregar  Não Carregar   
8   Não Carregar  Não Carregar      Carregar  Não Carregar  Não Carregar   
9   Não Carregar      Carregar      Carregar      Carregar  Não Carregar   
10  Não Carregar  Não Carregar  Não Carregar  Não Carregar  Não Carregar   
11  Não Carregar  N